In [1]:
import re
import string
import gensim
import logging

from gensim import corpora, models
from gensim.utils import smart_open, simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [2]:
allreviews = list()
review_details = dict()
clearing = 0

with open('Cell_Phones_&_Accessories.txt') as f:
    line = f.readlines()

lines = len(line)

f = open('Cell_Phones_&_Accessories.txt')
while lines > 0:
    line = f.readline()
    stripped_line = re.split(': ',line.strip())
    if len(stripped_line) > 1:
        review_details[stripped_line[0]] = stripped_line[1]
    elif len(stripped_line) == 1 and len(stripped_line[0]) > 0:
        review_details[stripped_line[0]] = ''
    elif len(stripped_line[0]) == 0:
        allreviews.append(review_details)
        review_details = dict()
        clearing = clearing + 1
    lines = lines - 1

In [3]:
len(allreviews)
# should be 78930

78930

In [4]:
#convert into list of list's, where each list is a review text

s = "string. With. Punctuation"
regex = re.compile('[%s]' % re.escape(string.punctuation))

def test_re(s):
    return regex.sub('', s)

def process(foo):
    foo = foo.lower()
    [token for token in simple_preprocess(foo) if token not in STOPWORDS]
    foo = test_re(foo)
    return foo

document = list()

for reviews in allreviews:
    tmp = list()
    tmp.append(process(reviews["review/text"]).split())
    document.extend(tmp)

In [5]:
len(document)

78930

In [6]:
'''def do_lda():
    dictionary = corpora.Dictionary(document)
    corpus = [dictionary.doc2bow(text) for text in document]
    lda = gensim.models.ldamulticore.LdaMulticore(corpus, id2word=dictionary, num_topics = 500, workers=25, passes=4)
    #lda_model = lda.print_topics(num_topics=25, num_words=5)
    return lda

%time lda_model = do_lda()
type(lda_model)'''

dictionary = corpora.Dictionary(document)

In [7]:
corpus = [dictionary.doc2bow(text) for text in document]

In [8]:
%time lda = models.ldamulticore.LdaMulticore(corpus, id2word=dictionary, num_topics = 10, workers=25, passes=4)

CPU times: user 1min 30s, sys: 12.5 s, total: 1min 43s
Wall time: 1min 34s


In [11]:
lda.print_topics(-1)

[(0,
  '0.070*the + 0.032*i + 0.027*it + 0.026*and + 0.022*to + 0.020*is + 0.019*this + 0.016*a + 0.013*my + 0.012*case'),
 (1,
  '0.015*and + 0.010*the + 0.009*it + 0.009*to + 0.008*for + 0.008*i + 0.008*my + 0.007*this + 0.006*is + 0.006*de'),
 (2,
  '0.023*phone + 0.019*to + 0.018*is + 0.017*this + 0.017*and + 0.017*the + 0.017*a + 0.016*i + 0.014*it + 0.012*you'),
 (3,
  '0.066*the + 0.030*is + 0.023*and + 0.022*a + 0.020*it + 0.020*to + 0.019*of + 0.017*phone + 0.016*this + 0.015*you'),
 (4,
  '0.056*the + 0.045*i + 0.032*it + 0.025*a + 0.021*to + 0.020*and + 0.019*is + 0.016*this + 0.014*have + 0.013*of'),
 (5,
  '0.036*the + 0.019*i + 0.018*a + 0.018*it + 0.014*and + 0.014*is + 0.012*to + 0.012*in + 0.012*for + 0.011*that'),
 (6,
  '0.054*the + 0.034*it + 0.032*to + 0.031*and + 0.023*a + 0.022*i + 0.019*phone + 0.016*of + 0.014*is + 0.013*for'),
 (7,
  '0.057*the + 0.036*i + 0.031*to + 0.030*and + 0.028*a + 0.020*is + 0.016*my + 0.016*it + 0.014*for + 0.014*this'),
 (8,
  '0.045

In [17]:
topics = lda.print_topics(num_topics=25, num_words=5)

In [15]:
%time tfidf_model = models.TfidfModel(corpus, id2word=dictionary)

CPU times: user 1.43 s, sys: 189 ms, total: 1.62 s
Wall time: 1.62 s


In [16]:
%time lsi_model = models.LsiModel(tfidf_model[corpus], id2word=dictionary, num_topics=200)

CPU times: user 6min 33s, sys: 5.18 s, total: 6min 39s
Wall time: 6min 38s


In [18]:
%time corpora.MmCorpus.serialize('reviewcorpus_tfidf.mm', tfidf_model[corpus])
%time corpora.MmCorpus.serialize('reviewcorpus_lsa.mm', lsi_model[tfidf_model[corpus]])

CPU times: user 20.7 s, sys: 404 ms, total: 21.1 s
Wall time: 21.3 s
CPU times: user 1min 3s, sys: 907 ms, total: 1min 4s
Wall time: 1min 4s


In [19]:
tfidf_corpus = corpora.MmCorpus('reviewcorpus_tfidf.mm')
# `tfidf_corpus` is now exactly the same as `tfidf_model[wiki_corpus]`
print(tfidf_corpus)

lsi_corpus = corpora.MmCorpus('reviewcorpus_lsa.mm')
# and `lsi_corpus` now equals `lsi_model[tfidf_model[wiki_corpus]]` = `lsi_model[tfidf_corpus]`
print(lsi_corpus)

MmCorpus(78930 documents, 122425 features, 4811390 non-zero entries)
MmCorpus(78930 documents, 200 features, 15786000 non-zero entries)


In [22]:
text = "I liked this phone very much. Battery dies very fast. Cheap yet powerful phone. \
Camera is not very good. But games run very fast."

# transform text into the bag-of-words space
bow_vector = dictionary.doc2bow(process(text).split())
print([(dictionary[id], count) for id, count in bow_vector])

[('this', 1), ('but', 1), ('i', 1), ('phone', 2), ('is', 1), ('not', 1), ('much', 1), ('very', 4), ('camera', 1), ('cheap', 1), ('yet', 1), ('good', 1), ('battery', 1), ('run', 1), ('fast', 2), ('liked', 1), ('games', 1), ('powerful', 1), ('dies', 1)]


In [23]:
# transform into LDA space
lda_vector = lda[bow_vector]
print(lda_vector)
# print the document's single most prominent LDA topic
print(lda.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

[(2, 0.26363101138675304), (8, 0.70436138829554928)]
0.045*the + 0.032*it + 0.032*a + 0.023*battery + 0.023*i + 0.023*phone + 0.017*and + 0.016*is + 0.015*to + 0.014*this


In [25]:
# store all trained models to disk
lda.save('reviewcorpus_lda.model')
lsi_model.save('reviewcorpus_lsi.model')
tfidf_model.save('reviewcorpus_tfidf.model')
dictionary.save('reviewcorpus.dictionary')

In [26]:
# load the same model back; the result is equal to `lda_model`
#same_lda_model = models.LdaModel.load('reviewcorpus_lda.model')

### Testing with a few first reviews. The entire corpus was used to train the models. Results may not be accurate, thus.

In [35]:
ASPECTS = set(["battery", "camera", "display", "service", "body", "price", "sound"])
i = 0
for reviews in allreviews:
    bow_vector = dictionary.doc2bow(process(reviews["review/text"]).split())
    lda_vector = lda[bow_vector]
    topics = lda.print_topic(max(lda_vector, key=lambda item: item[1])[0])
    words = topics.split(" + ")
    topics_for_this_review = dict()
    for word in words:
        prob = word.split("*")
        if prob[1] in ASPECTS:
            topics_for_this_review[prob[1]] = prob[0]
    if len(topics_for_this_review) > 0:
        print(topics_for_this_review)
    i = i +1
    if i == 1000:
        break

{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '0.023'}
{'battery': '

#### Finally, some reviews relating to aspect "battery" from top 1000 reviews from the dataset.